# Video Processing for BT Lab
This notebook extracts metadata from video files 

<br>

![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import pandas as pd

In [2]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [3]:
# patient alias id
ALIAS_ID = {
  "Rachael B" : "1",
  "Sara" : "2",
  "Kate" : "3",
  "Nick" : "4",
  "Daniel" : "5",
  "Illia" : "6",
  "Melissa" : "7",
  "Karlii" : "8",
  "Eddy" : "9",
  "John McNeil" : "10",
  "Farooq" : "11",
  "Krystyn" : "12",
  "Jay" : "13",
  "Frank" : "14",
  "Arun" : "15",
  "Riyan" : "16",
  "Joseph" : "17",
  "John Brunton" : "18",
  "Illia2" : "19",
  "Frank2" : "20"
}

In [4]:
def get_video_properties(mp4_file: str) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        old_fps = video.get(cv2.CAP_PROP_FPS)
        number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
        length = number_of_frames / old_fps
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video.release()
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    
    return video_properties

In [5]:
def get_alias(patient_info:dict, camera:str) -> str:
    if camera == "rgb":
        return ALIAS_ID.get(patient_info["ID"], "?")
    
    if camera == "thermal":
        return patient_info["ID"]

In [6]:
def add_video_properties(metadata:dict, camera:str) -> dict:
    new_patient_info = {}
    
    for patient_id, patient_info in metadata.items():
        try:
            video_path = patient_info["local path"]
            frames, fps, length_in_seconds = get_video_properties(video_path)
            patient_info["frames"] = frames
            patient_info["old fps"] = fps
            patient_info["length"] = length_in_seconds
            patient_info["alias"] = get_alias(patient_info, camera)
            new_patient_info[patient_id] = patient_info
        except Exception as e:
            print(f"{type(e)} - {patient_info['ID']}")

    return new_patient_info 

In [7]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [8]:
def export_to_excel(patient_info:dict, filename: str) -> None:
    patient_df = pd.DataFrame.from_dict(patient_info).transpose()
    patient_df = patient_df.drop(columns=["local path"])
    new_df = patient_df.set_index(["ID", "distance", "breathing", "blanket", "filename"])
    new_df.to_excel(filename)
    return new_df

In [9]:
def export_to_json(filename: str, all_patient_info: dict) -> None:
    patient_json = json.dumps(all_patient_info, indent=2)

    with open(filename, "w") as json_data:
        json_data.write(patient_json)

## Get video metadata and export to xlsx file (stage 2.1)

In [10]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb.json")
metadata_thermal = load_json(json_dir, "thermal.json")

In [11]:
""" extract metadata from video (rgb) """
complete_metadata_rgb = add_video_properties(metadata_rgb, "rgb")

In [12]:
""" save to json (rgb) """
json_rgb_complete = repo_dir + "/records/JSON/rgb_complete.json"
export_to_json(json_rgb_complete, complete_metadata_rgb)

In [13]:
""" patient info to pandas DataFrame (rgb)"""
rgb_xlsx = repo_dir + "/records/xlsx/rgb.xlsx"
rgb_df = export_to_excel(complete_metadata_rgb, rgb_xlsx)

In [14]:
""" extract metadata from video (thermal) """
complete_metadata_thermal = add_video_properties(metadata_thermal, "thermal")

In [15]:
""" save to json (thermal) """
json_thermal_complete = repo_dir + "/records/JSON/thermal_complete.json"
export_to_json(json_thermal_complete, complete_metadata_thermal)

In [16]:
""" patient info to pandas DataFrame (thermal)"""
thermal_xlsx = repo_dir + "/records/xlsx/thermal.xlsx"
thermal_df = export_to_excel(complete_metadata_thermal, thermal_xlsx)

In [17]:
""" read exported excel (rgb) """
new_rgb_df = pd.read_excel(rgb_xlsx)
new_rgb_df.describe()

,frames,length,old fps,alias
count,137.000000,137.000000,137.000000,137.000000
mean,1833.284672,37.508788,49.660760,9.678832
std,902.684933,18.801801,7.451798,5.372772
min,143.000000,4.113682,18.563774,1.000000
25%,1398.000000,26.896963,44.783596,5.000000
50%,1555.000000,32.738973,50.544559,9.000000
75%,2229.000000,46.704064,54.651671,14.000000
max,8111.000000,161.407619,60.460675,20.000000


In [18]:
""" read exported excel (thermal) """
new_thermal_df = pd.read_excel(thermal_xlsx)
new_thermal_df.describe()

,ID,frames,length,old fps,alias
count,15.000000,138.000000,138.000000,138.000000,138.000000
mean,8.000000,2272.442029,168.742995,15.000002,9.079710
std,4.472136,1466.445740,117.011025,9.001220,4.366531
min,1.000000,0.000000,0.000000,5.000000,1.000000
25%,4.500000,1499.750000,86.708333,10.000000,6.000000
50%,8.000000,2018.000000,175.550000,10.000000,9.000000
75%,11.500000,3565.750000,207.650000,30.000000,13.000000
max,15.000000,6450.000000,504.800000,30.000300,15.000000
